In [1]:
numberOfNodes=2
import mpi4py
import os
import ipyparallel as ipp

In [2]:
cluster = ipp.Cluster(engines="mpi", n=numberOfNodes,controller_ip='*')
rc = cluster.start_and_connect_sync()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [3]:
rc.wait_for_engines(n=numberOfNodes)
dview=rc[:]
rc.ids

[0, 1]

In [4]:
%%px --target 0:2
#%%px --target 0:1
import os, socket
#print(os.getpid())
print(socket.gethostname())

[stdout:0] x3100c0s21b0n0


[stdout:1] x3101c0s13b0n0


In [5]:
%%px
from mpi4py import MPI

useGPU=False

if useGPU:
    import cupy as cp
else:
    import numpy as cp

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    N=2
    data = cp.random.randn(N,N,N,N)
    comm.send(data, dest=1, tag=42)
    print('Process {} sent data:'.format(rank), data)
    
elif rank == 1:
    data = comm.recv(source=0, tag=42)
    print('Process {} received data:'.format(rank), data)

[stdout:0] Process 0 sent data: [[[[-0.00705688  0.1574703 ]
   [-0.11614565 -1.36530107]]

  [[ 1.9377235   0.19872024]
   [ 0.20897026  1.86760844]]]


 [[[-1.19135132  1.44246809]
   [-1.49924247  0.45127102]]

  [[ 1.77763836  0.49243783]
   [-0.4570443   0.96459175]]]]


[stdout:1] Process 1 received data: [[[[-0.00705688  0.1574703 ]
   [-0.11614565 -1.36530107]]

  [[ 1.9377235   0.19872024]
   [ 0.20897026  1.86760844]]]


 [[[-1.19135132  1.44246809]
   [-1.49924247  0.45127102]]

  [[ 1.77763836  0.49243783]
   [-0.4570443   0.96459175]]]]
